# Explaination

Here we're gonna randomly sample 100 of each feature type from the val set, then add in extras to get to 400 panos total.

In [1]:
import csv
from collections import defaultdict
import numpy
import os
import random

In [12]:
#repo_path = '/Users/galenweld/project_sidewalk_ml'
repo_path = '/mnt/c/Users/gweld/sidewalk/sidewalk_ml'
#path_to_labels    = os.path.join(repo_path, 'visualizations/minus_onboard.csv')
path_to_labels    = os.path.join(repo_path, '../minus_onboard.csv')
path_to_train_set = os.path.join(repo_path, 'dataset_csvs/Train.csv')
path_to_val_set   = os.path.join(repo_path, 'dataset_csvs/Val.csv')

In [7]:
with open(path_to_train_set) as f:
    reader = csv.reader(f)
    header_row = next(reader)
print " ".join(header_row)

Pano ID SV_x SV_y Label Photographer Heading Heading Label ID


In [8]:
train_panos = set()

with open(path_to_train_set) as f:
    reader = csv.reader(f)
    for row in reader:
        pano_id = row[0]
        train_panos.add(pano_id)
train_panos.remove("Pano ID")

In [9]:
val_panos = set()

with open(path_to_val_set) as f:
    reader = csv.reader(f)
    for row in reader:
        pano_id = row[0]
        val_panos.add(pano_id)
val_panos.remove("Pano ID")

In [10]:
len(train_panos), len(val_panos)

(46463, 5774)

In [13]:
panos_by_label = defaultdict(set)

with open(path_to_labels) as f:
    reader = csv.reader(f)
    for row in reader:
        pano_id = row[0]
        label = int(row[3])
        if label in (1,2,3,4) and pano_id in val_panos:
            panos_by_label[label].add(pano_id)

print "{:^6}{}".format("Label", "Num Panos")
for label, panos in panos_by_label.items():
    print "{:^6}{}".format(label, len(panos))

Label Num Panos
  1   4075
  2   1106
  3   1417
  4   685


In [31]:
sampled_panos = {}

for label, panos in panos_by_label.items():
    s = set()
    while len(s) < 100:
        num_needed = 100- len(s)
        s |= set( random.sample(panos, num_needed) )
    sampled_panos[label] = s

print "{:^6}{}".format("Label", "Num Panos")
for label, panos in sampled_panos.items():
    print "{:^6}{}".format(label, len(panos))

Label Num Panos
  1   100
  2   100
  3   100
  4   100


In [36]:
all_samples = set()
for s in sampled_panos.values():
    all_samples |= s
print len(all_samples)

396


In [37]:
while len(all_samples) < 400:
    all_samples |= set( random.sample(panos_by_label[1],1) )
print len(all_samples)

400


In [39]:
with open('sampled_panos_for_ground_truth.txt', 'w') as f:
    for pano_id in all_samples:
        f.writelines(pano_id+'\n')

# Going back to get panos with most Missing Curb Ramps

In [14]:
val_panos_missing_ramps = defaultdict(int)

with open(path_to_labels) as f:
    reader = csv.reader(f)
    for row in reader:
        pano_id = row[0]
        label = int(row[3])
        if label == 2 and pano_id in val_panos:
            val_panos_missing_ramps[pano_id] += 1
            

In [17]:
for pid, count in sorted(val_panos_missing_ramps.items(), key=lambda x: x[1], reverse=True)[:20]:
    print pid, count

17Kf6DFklN_pCy36L5OOUw 18
bjkfY9Oa3jV3B-g8o2p4Uw 18
WSkAfEpA_OtK5u4T57ykTQ 16
arbGR9mQ_Gbr-LNWio3iKw 13
2Lzx4aEfsCmiX1dlyr7DNQ 10
v1L4iS6mB7ZndEroZRyHww 10
ZQdiE5shETg2os-kciAHqA 10
Ju-5ZGopQtmIcvwhciFbIw 7
M_ob8iOynti4Vy8XQmnClg 7
U1EwgzvToBS4kXB_hTV8WQ 7
TNUrQtVnmTG99hJuSwDYHg 6
l5eYxTKkeNVRZjg6NkuHpQ 6
oWiYpE-qlIm4xv7ZtjPh9Q 6
kSh8CSVzsV2q3IAWBpSjoA 6
xQ7UxceNjMkmfUbawDHkyA 6
stw2DE74Z6T7UiVFg7Vt-Q 6
IWSfloST5O5Cc3OpV1xqag 6
yGVUb9aH5ENvW5ZfMy4a8Q 6
GXruCkxCjHAWUDqcmHfAxw 6
rSfND_SnSBwCuUubAOduxQ 6


In [26]:
already_sampled = set()
with open('sampled_panos_for_ground_truth.txt') as f:
    for line in f:
        already_sampled.add(line[:-1])
        
print "Counted {} already sampled panos.".format(len(already_sampled))

Counted 402 already sampled panos.


In [37]:
with open('new_panos_with_missing_ramps.txt', 'w') as f:
    for pid, count in sorted(val_panos_missing_ramps.items(), key=lambda x: x[1], reverse=True)[:100]:
        if pid in already_sampled:
            print "skipping already sampled pano"
            continue
        print pid, count
        f.writelines(pid+'\n')

skipping already sampled pano
bjkfY9Oa3jV3B-g8o2p4Uw 18
WSkAfEpA_OtK5u4T57ykTQ 16
arbGR9mQ_Gbr-LNWio3iKw 13
2Lzx4aEfsCmiX1dlyr7DNQ 10
v1L4iS6mB7ZndEroZRyHww 10
ZQdiE5shETg2os-kciAHqA 10
Ju-5ZGopQtmIcvwhciFbIw 7
M_ob8iOynti4Vy8XQmnClg 7
U1EwgzvToBS4kXB_hTV8WQ 7
TNUrQtVnmTG99hJuSwDYHg 6
l5eYxTKkeNVRZjg6NkuHpQ 6
oWiYpE-qlIm4xv7ZtjPh9Q 6
kSh8CSVzsV2q3IAWBpSjoA 6
xQ7UxceNjMkmfUbawDHkyA 6
skipping already sampled pano
IWSfloST5O5Cc3OpV1xqag 6
yGVUb9aH5ENvW5ZfMy4a8Q 6
GXruCkxCjHAWUDqcmHfAxw 6
rSfND_SnSBwCuUubAOduxQ 6
C2ArtRTlgvVJswjb_egxeQ 6
7Nm83eNWhDpo1Au27Qw1qA 6
1-2MNgRG-iN6DwktcIRE_g 6
skipping already sampled pano
R7oO4MPA6YpZ1DmLo2PVlg 6
2qo8E-Wboo329orUetqzAQ 6
uYR2NZpjnSuHRo9WXozX7g 6
_ksfAI3JAieAsCq4uGGUYw 6
2u2OFTh_9QFMVM2umyJ-cQ 5
BX-IlV8IsOsA2RGm7YhRFQ 5
skipping already sampled pano
da6e7v5cS5EfUQiUmpkCSg 5
kEe57krKXVdJSB_epVDnMA 5
3fNxkHJU1uA0qRWD91QGJw 5
skipping already sampled pano
skipping already sampled pano
03nLDyiGmStaf5UsEY9TMQ 5
j72ac6giHtyulFl-xH6I5g 5
zKlyR-dpvLhFqq

In [38]:
for pid, count in sorted(val_panos_missing_ramps.items(), key=lambda x: x[1], reverse=True)[100:]:
        if pid in already_sampled:
            continue
        print pid

JQs56aaDjLlHH9CmLexvhA
bseRm4GdfpjP_xPOxwjcXA
BMDmthcD33SUkxmGS1uKHw
Gas0MihKF4Gpaxq5GmKQ9Q
FyedNmI0MqDfZ3mXxmCNAA
3dpo0OHzmq-qJydzROzQzQ
tBHb1pPhSpFxIWZBSwuyCA
p5evS0UbsShiuSvOBMZrbg
yVe1fcPx6mfDX250CLiFeQ
T0paMbYqSgQX6tBLWtVawQ
8y_OGb3OqiB8x19eEuC64Q
00l_WbrbGY7kyugIhpid4A
c0C3JWOBRvTt3e5LRTe3sQ
m5UVuG8mTK8pmKONRPFHwg
OmF9cCcl7ZuvkzofUItmlg
qbRKCQFTa8iIPJNURt-Mmg
BzMjQyj2V1ppvpPKLQFCoA
4zbNPNz4-aFgJ0GjBQJAYA
Elk349oF9ZtSiPzgujJNww
MlR9wn9kz-nWGx1m5syKGQ
9G9jmhFjYxfdy8TOCHNbmw
GNbAKyP229KshsIf4v3UQA
Mef_cmD7Y1bua1HNi0oNww
DD7j9_CpPQgWYn1wtJyS2g
VNub-RUTKS1FpHHa8fZbcA
U_1iuG4g_O4_jE1qHkLo1g
p13fEAtelHv3rpXzQ5xHGg
MV5MJfwSbhSNSoNrouezkA
CgyzcHD3zNxD-q_iAdOSFg
Qw_eBOIwV8TrFTNTUhV2_g
drJgSNWSfWknSuUpddyjJg
PH9agveT6oOAsQ_vGZZ6FA
uqh7__TVEw3LVGBAyx0-7A
vT0UBxtEnsqWdD2_tU7_BA
O1-QqGzzYN_KM6ogY01l5w
WEuKyN_z_XjCeaB-OCBZwg
mVR7qA385AW1xx8378a4SA
ZpPqt2RHiZHnHa_tcnpHMQ
JzrgovP3wBYZkzsv7aAIlA
ijNSI1omqRavp74L6qdWyg
xG7sLobybT64rI0grblYzg
wda7pncDotvpyIY_2hf_Dw
etwMvmgyUv3FSIHF7f6D6w
pnyseFm_He3